In [ ]:
# use this in google colab
# !pip install neuralprophet[live]
# !pip install torch
# !pip install holidays

In [ ]:
import pandas as pd

Uploading the prepared data to the dataframe from a csv-file.
Then changing the datatype of the 'date' column to 'date'.

In [ ]:
sales = pd.read_csv('data.csv', encoding='utf-8', sep=';')
sales.date = pd.to_datetime(sales.date)

Then we split the dataframe so the prediction will be done for each product separately.

In [ ]:
sales = sales.set_index('date').groupby('product')\
.resample('D').sum().reset_index().set_index('product')


And splitting the frame

In [ ]:
split_df = [d for _, d in sales.groupby('product')]
prophet_list = split_df.copy()

Both Prophet and NeuralProphet await dataframe with columns 'ds' and 'y' so we need to rename our columns and drop extra ones.
We will do it for each dataframe we got from splitting.

In [ ]:
i=0
while i < len(prophet_list):
    prophet_list[i] = prophet_list[i].dropna()
    prophet_list[i] = prophet_list[i].rename(columns={'date':'ds', 'roll':'y'})
    prophet_list[i] = prophet_list[i].reset_index().drop('product', axis=1)
    i+=1

Knowing that we possibly have a lot of products, we will try to make forecasts in parallel via torch.multiprocessing

NB! There's an issue considering torch multiprocessing: we need to import the NeuralProphet model from another file.
See here: https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
Or here: https://github.com/python/cpython/issues/69240


In [ ]:
import torch.multiprocessing as mp
import os
from forecast_sales import forecast_sales

In [ ]:
if __name__ == '__main__':
    with mp.Pool(os.cpu_count()) as p:
        result = list(p.imap(forecast_sales, prophet_list))
        p.close()
        p.join()

We create an empty frame to concat the results of Prophet.

In [ ]:
i=0
result_df = pd.DataFrame(columns=['ds', 'y', 'yhat1'])
while i < len(result):
    result_df = pd.concat([result_df, result[i]])
    i+=1

Finally, there's a glimpse of what we got (yhat - predicted value)

In [ ]:
result_df.tail()

,ds,y,yhat1
235,2023-04-23,NaN,2682.086182
236,2023-04-24,NaN,1901.761719
237,2023-04-25,NaN,2321.318115
238,2023-04-26,NaN,1947.991821
239,2023-04-27,NaN,2172.741699


In [ ]:
result_df.to_csv('current_prophet_result.csv', index=False, sep=';', encoding='utf-8')